In [1]:
import sys

import pandas as pd
import numpy as np
from pathlib import Path
import yaml
from typing import *

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, DataFrame, Window
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, space_eval
import itertools

from pyspark.ml.feature import VectorAssembler, MinMaxScaler, BucketedRandomProjectionLSH
from pyspark.ml import Pipeline
from pyspark.ml.functions import vector_to_array

from utils import *

pio.renderers.default = "iframe"

In [2]:
n_cores = 28

spark = (SparkSession.builder
         .master(f"local[{n_cores}]")
         .appName("Saturation")
         .config("spark.sql.shuffle.partitions", "500")
         .config("spark.driver.memory", "60g")
         .config("spark.driver.maxResultSize", "8g")
         .getOrCreate())

24/03/02 13:24:47 WARN Utils: Your hostname, muninn resolves to a loopback address: 127.0.1.1; using 192.168.86.20 instead (on interface enp8s0)
24/03/02 13:24:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/02 13:24:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/02 13:24:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/02 13:24:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [15]:
base_path = "/data/saturation/thesis_run_20240130"
r_stat = 5
study_region_size = 2000 * 2000 / r_stat ** 2

configs_df = create_configs_df(read_configs(base_path, spark))
data = spark.read.parquet(f"{base_path}/*/statistics_*.parquet")

# State c2c nn dist in terms of r_stat and log-scale
data = data.select(
    "*",
    F.log10(F.col("center_to_center_nearest_neighbor_distance_mean") / F.lit(r_stat)).alias("log_mean_c2c_nn_dist"),
    F.log10(F.col("n_craters_in_study_region")).alias("log_n_craters_in_study_region"),
    F.log10("n_craters_added_in_study_region").alias("log_n_craters_added_in_study_region"),
    (F.col("n_craters_in_study_region") / F.col("n_craters_added_in_study_region")).alias("information_remaining")
)

In [5]:
data.count()

444699515

In [16]:
data_subset = data.where(
    (F.col("n_craters_added_in_study_region") > F.lit(100))
).sample(0.0005)

df = join_configs(data_subset, configs_df, spark).toPandas()

In [7]:
configs_df.toPandas().sort_values("slope")

,effective_radius_multiplier,min_rim_percentage,r_stat_multiplier,simulation_id,slope
7003,1.467602,0.686809,10.148981,2164,1.000246
709,1.624211,0.604381,14.055626,1148,1.000381
1207,1.436503,0.392966,12.947683,5510,1.000389
7522,1.447905,0.740991,12.896511,1017,1.000441
689,1.583571,0.543519,6.508161,1142,1.000661
...,...,...,...,...,...
7443,1.369571,0.492816,13.385134,5501,2.999578
2243,1.877052,0.745182,13.677595,1925,2.999638
2081,1.655109,0.690670,6.200024,9648,2.999713
8468,1.783821,0.642164,9.567673,7919,2.999827


In [ ]:
def plot_log_N_and_log_NNd_for_simulation(simulation_id: int, stats_df: pd.DataFrame):
    df = stats_df[stats_df.simulation_id == simulation_id].sort_values("n_craters_added_in_study_region")
    df.head()
    
    print(df[["slope", "effective_radius_multiplier", "r_stat_multiplier", "min_rim_percentage"]].iloc[0])
    
    fig = plt.figure(figsize=(12, 3))
    ax1 = fig.add_subplot(111)
    
    plt1 = ax1.plot(df.n_craters_added_in_study_region, df.log_mean_c2c_nn_dist, color='red', label="$log(\overline{NN_{d}})$")
    ax1.set_ylabel("$log(\overline{NN_{d}})$")
    ax1.set_xlabel("N Craters Added")
    
    ax2 = ax1.twinx()
    plt2 = ax2.plot(df.n_craters_added_in_study_region, np.log10(df.n_craters_in_study_region), color="blue", label="$log(N_{obs})$")
    ax2.set_ylabel("$log(N_{obs})$")
    
    lns = plt1 + plt2
    ax1.legend(lns, [l.get_label() for l in lns], loc=1)
    
    ax1.set_title(f"Simulation {simulation_id}")

    plt.show()

In [ ]:
for sim_id in np.random.choice(df.simulation_id.drop_duplicates(), 20):
    plot_log_N_and_log_NNd_for_simulation(sim_id, df)

In [17]:
# Randomly arrange the points
# Ordering affects which points in the subsequent plots are "on top" of others
df["rnd"] = np.random.rand(df.shape[0])
df = df.sort_values("rnd")

In [39]:
df.columns

Index(['slope', 'min_rim_percentage', 'effective_radius_multiplier',
       'r_stat_multiplier', 'crater_id', 'n_craters_added_in_study_region',
       'n_craters_in_study_region', 'areal_density', 'areal_density_overlap_2',
       'areal_density_overlap_3',
       'center_to_center_nearest_neighbor_distance_mean',
       'center_to_center_nearest_neighbor_distance_stdev',
       'center_to_center_nearest_neighbor_distance_min',
       'center_to_center_nearest_neighbor_distance_max', 'radius_mean',
       'radius_stdev', 'z', 'za', 'simulation_id', 'log_mean_c2c_nn_dist',
       'log_n_craters_in_study_region', 'log_n_craters_added_in_study_region',
       'information_remaining', 'rnd'],
      dtype='object')

In [64]:
df["log_za"] = np.log10(df.za)
fig = px.scatter_3d(
    df,
    x="log_n_craters_in_study_region",
    y="log_n_craters_added_in_study_region",
    z="slope",
    color="log_n_craters_in_study_region",
    hover_data=["simulation_id", "slope", "n_craters_added_in_study_region", "areal_density", "n_craters_in_study_region"],
    size_max=1,
    width=1000,
    height=500
)
fig.update_layout(
    yaxis_title=dict(
        text="$log_{10}(N_{tot})$",
        font=dict(size=18)
    ),
    xaxis_title=dict(
        text="za",
        font=dict(size=18)
    ),
)
fig.update_traces(marker={"size":1})
fig.show()

/home/mason/code/saturation/venv_311/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning:

invalid value encountered in log10



In [66]:
# Trying a range selector
to_show = df.copy()
to_show["slope_selector"] = ((to_show.slope - 1) * 10).astype("int") / 10 + 1
to_show = to_show.sort_values("slope_selector")
to_show["index"] = range(to_show.shape[0])

range_x = [to_show.log_n_craters_in_study_region.min(), to_show.log_n_craters_in_study_region.max()]
range_y = [to_show.log_n_craters_added_in_study_region.min(), to_show.log_n_craters_added_in_study_region.max()]

fig = px.scatter(
    to_show,
    x="log_n_craters_in_study_region",
    y="log_n_craters_added_in_study_region",
    color="log_n_craters_in_study_region",
    hover_data=["slope", "n_craters_added_in_study_region"],
    size_max=1,
    animation_frame="slope_selector",
    animation_group="index",
    range_x=range_x,
    range_y=range_y,
    width=1000,
    height=500
)
fig.update_layout(
    xaxis_title=dict(
        text="$log_{10}(N_{obs})$",
        font=dict(size=18)
    ),
    yaxis_title=dict(
        text="$log(N_{tot})$",
        font=dict(size=18)
    ),
)
fig.update_traces(marker={"size":3})
fig.show()

In [52]:
fig = px.scatter(
    df,
    x="log_mean_c2c_nn_dist",
    y="log_n_craters_in_study_region",
    color="slope",
    hover_data=["simulation_id", "slope", "n_craters_added_in_study_region", "areal_density", "n_craters_in_study_region"],
    size_max=1,
    width=1000,
    height=500
)
fig.update_layout(
    xaxis_title=dict(
        text="$log_{10}(\overline{NN_d})$",
        font=dict(size=18)
    ),
    yaxis_title=dict(
        text="$log_{10}(N_{obs})$",
        font=dict(size=18)
    ),
)
fig.update_traces(marker={"size":3})
fig.show()

In [37]:
fig = px.scatter_3d(
    df,
    x="log_mean_c2c_nn_dist",
    y="log_n_craters_in_study_region",
    z="slope",
    color="slope",
    hover_data=["simulation_id", "slope", "n_craters_added_in_study_region", "areal_density", "n_craters_in_study_region"],
    size_max=1,
    width=1000,
    height=500
)
fig.update_layout(
    xaxis_title=dict(
        text="$log_{10}(\overline{NN_d})$",
        font=dict(size=18)
    ),
    yaxis_title=dict(
        text="$log_{10}(N_{obs})$",
        font=dict(size=18)
    ),
)
fig.update_traces(marker={"size":3})
fig.show()

In [63]:
fig = px.scatter(
    df,
    x="log_mean_c2c_nn_dist",
    y="log_n_craters_added_in_study_region",
    color="slope",
    hover_data=["simulation_id", "slope", "n_craters_added_in_study_region", "areal_density", "n_craters_in_study_region"],
    size_max=1,
    width=1600,
    height=600,
)
fig.update_layout(
    xaxis_title=dict(
        text="$log_{10}(\overline{NN_d}})$",
        font=dict(size=18)
    ),
    yaxis_title=dict(
        text="$log_{10}(N_{tot})$",
        font=dict(size=18)
    ),
)
fig.update_traces(marker={"size":3})
fig.show()

In [36]:
# Trying a range selector
to_show = df.copy()
to_show["slope_selector"] = ((to_show.slope - 1) * 10).astype("int") / 10 + 1
to_show = to_show.sort_values("slope_selector")
to_show["index"] = range(to_show.shape[0])

range_x = [to_show.log_mean_c2c_nn_dist.min(), to_show.log_mean_c2c_nn_dist.max()]
range_y = [to_show.log_n_craters_added_in_study_region.min(), to_show.log_n_craters_added_in_study_region.max()]

fig = px.scatter(
    to_show,
    x="log_mean_c2c_nn_dist",
    y="log_n_craters_added_in_study_region",
    color="log_n_craters_in_study_region",
    hover_data=["slope", "n_craters_added_in_study_region"],
    size_max=1,
    animation_frame="slope_selector",
    animation_group="index",
    range_x=range_x,
    range_y=range_y,
    width=1000,
    height=500
)
fig.update_layout(
    xaxis_title=dict(
        text="$log_{10}(N_{obs}})$",
        font=dict(size=18)
    ),
    yaxis_title=dict(
        text="$log(N_{tot})$",
        font=dict(size=18)
    ),
)
fig.update_traces(marker={"size":3})
fig.show()

In [22]:
fig = px.scatter(
    df,
    x="log_n_craters_in_study_region",
    y="log_n_craters_added_in_study_region",
    color="slope",
    hover_data=["simulation_id", "slope", "n_craters_added_in_study_region", "areal_density", "n_craters_in_study_region"],
    size_max=1,
    width=1600,
    height=600,
)
fig.update_layout(
    xaxis_title=dict(
        text="$log_{10}(N_{obs}})$",
        font=dict(size=18)
    ),
    yaxis_title=dict(
        text="$log_{10}(N_{tot})$",
        font=dict(size=18)
    ),
)
fig.update_traces(marker={"size":3})
fig.show()

In [34]:
# Comparing N_tot to N_obs
# Trying a range selector
to_show = df.copy()
to_show["slope_selector"] = ((to_show.slope - 1) * 10).astype("int") / 10 + 1
to_show = to_show.sort_values("slope_selector")
to_show["index"] = range(to_show.shape[0])

range_x = [to_show.log_n_craters_in_study_region.min(), to_show.log_n_craters_in_study_region.max()]
range_y = [to_show.log_n_craters_added_in_study_region.min(), to_show.log_n_craters_added_in_study_region.max()]

fig = px.scatter(
    to_show,
    x="log_n_craters_in_study_region",
    y="log_n_craters_added_in_study_region",
    color="log_mean_c2c_nn_dist",
    hover_data=["slope", "n_craters_added_in_study_region"],
    size_max=1,
    animation_frame="slope_selector",
    animation_group="index",
    range_x=range_x,
    range_y=range_y,
    width=1000,
    height=500
)
fig.update_layout(
    xaxis_title=dict(
        text="$log_{10}(N_{obs}})$",
        font=dict(size=18)
    ),
    yaxis_title=dict(
        text="$log(N_{tot})$",
        font=dict(size=18)
    ),
)
fig.update_traces(marker={"size":3})
fig.show()

## Plotting single simulations

In [ ]:
df = data.where(
    (F.col("n_craters_added_in_study_region") > F.lit(25))
    & (data.simulation_id == 4935)
).cache()

n_pts = 10000
n_obs = df.count()

if n_obs > n_pts:
    sample_fraction = n_pts / n_obs
    df = df.sample(sample_fraction)

pandas_df = df.toPandas()
pandas_df["log_n_craters_in_study_region"] = np.log10(pandas_df.n_craters_in_study_region)
pandas_df = pandas_df.sort_values("log_n_craters_in_study_region")

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(
    # np.log10(pandas_df.n_craters_added_in_study_region),
    np.log10(pandas_df.log_n_craters_in_study_region),
    pandas_df.log_mean_c2c_nn_dist,    
)
plt.title("Single Simulation, b=3")
plt.xlabel("$log_{10}(N_{tot})$", size=14)
plt.ylabel("$log_{10}(\overline{NN_d})$", size=14)
plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(
    np.log10(pandas_df.n_craters_added_in_study_region),
    np.log10(pandas_df.n_craters_in_study_region),
)
plt.title("Single Simulation, b=3")
plt.xlabel("$log_{10}(N_{tot})$", size=14)
plt.ylabel("$log_{10}(N_obs)$", size=14)
plt.show()